# QSVC with Custom Feature Map - Flood Risk Dataset

This notebook implements a Quantum Support Vector Classifier (QSVC) with a custom feature map:
- 3 qubits for 13 features (dimensionality reduction)
- Rotational Y (RY) gates applied to all qubits
- Efficient SU2 gates applied between all qubits for entanglement

We compare:
1. Classical SVC
2. QSVC with ZZ Feature Map
3. QSVC with Custom Feature Map

Data sizes tested: 50, 250, 10000 rows


## 1. Imports and Setup


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
algorithm_globals.random_seed = 42
np.random.seed(42)


## 2. Data Loading and Preprocessing


In [ ]:
# Load and preprocess data
df = pd.read_csv("flood_risk_dataset_india 2.csv")

# Encode categorical variables
le_land = LabelEncoder()
le_soil = LabelEncoder()
df['Land Cover'] = le_land.fit_transform(df['Land Cover'])
df['Soil Type'] = le_soil.fit_transform(df['Soil Type'])

# Separate features and target
X = df.drop('Flood Occurred', axis=1).values
y = df['Flood Occurred'].values

print(f"Dataset shape: {X.shape}")
print(f"Features: {X.shape[1]}")
print(f"Classes: {np.unique(y)}")
print(f"Class distribution: {np.bincount(y)}")


## 3. Custom Feature Map Definition


In [ ]:
# Custom Feature Map: 3 qubits, RY gates, Efficient SU2 entanglement
def create_custom_feature_map(num_qubits=3, feature_dimension=3, reps=2):
    """
    Creates a custom feature map with:
    - 3 qubits for 3 features (after dimensionality reduction from 13)
    - RY gates on all qubits
    - Efficient SU2 gates between all qubits for entanglement
    
    Args:
        num_qubits: Number of qubits (3)
        feature_dimension: Number of input features (3, after PCA reduction)
        reps: Number of repetition layers
    
    Returns:
        QuantumCircuit: Custom feature map circuit
    """
    # Create parameter vector for features
    # We use 3 features, so we need 3 parameters
    feature_params = ParameterVector('x', length=feature_dimension)
    
    qc = QuantumCircuit(num_qubits)
    
    for rep in range(reps):
        # Apply RY gates to all qubits using the 3 input features
        for qubit in range(num_qubits):
            # Use feature parameters cyclically if needed
            feature_idx = qubit % feature_dimension
            qc.ry(feature_params[feature_idx], qubit)
        
        # Apply efficient SU2 gates between all qubits for entanglement
        # SU2 is a 2-qubit parameterized gate
        # We implement it using parameterized rotations and CNOT
        for i in range(num_qubits):
            for j in range(i + 1, num_qubits):
                # Parameterized SU2 gate implementation
                # Use feature parameters for rotation angles
                # Combine features to create entanglement parameters
                theta_i = feature_params[i % feature_dimension]
                theta_j = feature_params[j % feature_dimension]
                # Use a combination of features for the entangling parameter
                phi = (feature_params[0] + feature_params[1] + feature_params[2]) / 3
                
                # Efficient SU2 gate: parameterized rotations + entangling CNOT
                qc.ry(theta_i, i)
                qc.ry(theta_j, j)
                qc.cx(i, j)
                qc.ry(phi, j)
                qc.cx(i, j)
    
    return qc

# Create the custom feature map
custom_feature_map = create_custom_feature_map(num_qubits=3, feature_dimension=3, reps=2)
print("Custom Feature Map Circuit:")
print(custom_feature_map.draw(output='text'))


## 4. Helper Functions for Feature Processing


In [ ]:
# Function to reduce 13 features to 3 dimensions for 3-qubit encoding
def reduce_features(X, target_dim=3):
    """
    Reduces 13 features to 3 dimensions using PCA
    """
    from sklearn.decomposition import PCA
    pca = PCA(n_components=target_dim)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Function to normalize features to [0, 2π] for quantum encoding
def normalize_for_quantum(X):
    """
    Normalize features to [0, 2π] range for quantum gates
    """
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)
    X_range = X_max - X_min
    X_range[X_range == 0] = 1  # Avoid division by zero
    X_norm = 2 * np.pi * (X - X_min) / X_range
    return X_norm


## 5. Experiment Setup


In [ ]:
# Data sizes to test
data_sizes = [50, 250, 10000]

# Store results
results = {
    'data_size': [],
    'classical_svc_accuracy': [],
    'classical_svc_time': [],
    'qsvc_zz_accuracy': [],
    'qsvc_zz_time': [],
    'qsvc_custom_accuracy': [],
    'qsvc_custom_time': []
}

print("Starting experiments with different data sizes...")
print("=" * 80)


## 6. Model Training and Evaluation

This section trains and evaluates three models across different data sizes:
- **Classical SVC**: Traditional Support Vector Classifier with RBF kernel
- **QSVC with ZZ Feature Map**: Quantum SVC using the standard ZZ feature map
- **QSVC with Custom Feature Map**: Quantum SVC using our custom feature map with RY gates and SU2 entanglement


In [ ]:
for size in data_sizes:
    print(f"\n{'='*80}")
    print(f"Testing with {size} rows of data")
    print(f"{'='*80}")
    
    # Sample data (if size is less than total dataset)
    if size <= len(X):
        indices = np.random.choice(len(X), size, replace=False)
        X_sample = X[indices]
        y_sample = y[indices]
    else:
        X_sample = X
        y_sample = y
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_sample, y_sample, test_size=0.2, random_state=42, stratify=y_sample
    )
    
    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # ========== 1. Classical SVC ==========
    print(f"\n[1/3] Training Classical SVC...")
    start_time = time.time()
    classical_svc = SVC(kernel='rbf', random_state=42)
    classical_svc.fit(X_train_scaled, y_train)
    classical_pred = classical_svc.predict(X_test_scaled)
    classical_acc = accuracy_score(y_test, classical_pred)
    classical_time = time.time() - start_time
    
    print(f"  Accuracy: {classical_acc:.4f}")
    print(f"  Time: {classical_time:.2f} seconds")
    
    results['data_size'].append(size)
    results['classical_svc_accuracy'].append(classical_acc)
    results['classical_svc_time'].append(classical_time)
    
    # ========== 2. QSVC with ZZ Feature Map ==========
    print(f"\n[2/3] Training QSVC with ZZ Feature Map...")
    
    # Reduce features to 3 for quantum encoding
    X_train_reduced, pca_zz = reduce_features(X_train_scaled, target_dim=3)
    X_test_reduced = pca_zz.transform(X_test_scaled)
    
    # Normalize for quantum
    X_train_q = normalize_for_quantum(X_train_reduced)
    X_test_q = normalize_for_quantum(X_test_reduced)
    
    # Create ZZ feature map (3 qubits)
    zz_feature_map = ZZFeatureMap(feature_dimension=3, reps=2)
    
    # Create quantum kernel
    zz_kernel = FidelityQuantumKernel(feature_map=zz_feature_map)
    
    start_time = time.time()
    qsvc_zz = QSVC(quantum_kernel=zz_kernel)
    qsvc_zz.fit(X_train_q, y_train)
    qsvc_zz_pred = qsvc_zz.predict(X_test_q)
    qsvc_zz_acc = accuracy_score(y_test, qsvc_zz_pred)
    qsvc_zz_time = time.time() - start_time
    
    print(f"  Accuracy: {qsvc_zz_acc:.4f}")
    print(f"  Time: {qsvc_zz_time:.2f} seconds")
    
    results['qsvc_zz_accuracy'].append(qsvc_zz_acc)
    results['qsvc_zz_time'].append(qsvc_zz_time)
    
    # ========== 3. QSVC with Custom Feature Map ==========
    print(f"\n[3/3] Training QSVC with Custom Feature Map...")
    
    # Use the same reduced features
    # Normalize for quantum (already done above, but ensure consistency)
    X_train_custom = normalize_for_quantum(X_train_reduced)
    X_test_custom = normalize_for_quantum(X_test_reduced)
    
    # Create custom feature map
    custom_feature_map = create_custom_feature_map(num_qubits=3, feature_dimension=3, reps=2)
    
    # Create quantum kernel with custom feature map
    custom_kernel = FidelityQuantumKernel(feature_map=custom_feature_map)
    
    start_time = time.time()
    qsvc_custom = QSVC(quantum_kernel=custom_kernel)
    qsvc_custom.fit(X_train_custom, y_train)
    qsvc_custom_pred = qsvc_custom.predict(X_test_custom)
    qsvc_custom_acc = accuracy_score(y_test, qsvc_custom_pred)
    qsvc_custom_time = time.time() - start_time
    
    print(f"  Accuracy: {qsvc_custom_acc:.4f}")
    print(f"  Time: {qsvc_custom_time:.2f} seconds")
    
    results['qsvc_custom_accuracy'].append(qsvc_custom_acc)
    results['qsvc_custom_time'].append(qsvc_custom_time)
    
    print(f"\nSummary for {size} rows:")
    print(f"  Classical SVC:     {classical_acc:.4f} ({classical_time:.2f}s)")
    print(f"  QSVC (ZZ):         {qsvc_zz_acc:.4f} ({qsvc_zz_time:.2f}s)")
    print(f"  QSVC (Custom):     {qsvc_custom_acc:.4f} ({qsvc_custom_time:.2f}s)")


## 7. Results Summary


In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)
print(results_df.to_string(index=False))


## 8. Visualizations


In [ ]:
# Visualization: Accuracy Comparison
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(results_df['data_size'], results_df['classical_svc_accuracy'], 
         marker='o', label='Classical SVC', linewidth=2)
plt.plot(results_df['data_size'], results_df['qsvc_zz_accuracy'], 
         marker='s', label='QSVC (ZZ Feature Map)', linewidth=2)
plt.plot(results_df['data_size'], results_df['qsvc_custom_accuracy'], 
         marker='^', label='QSVC (Custom Feature Map)', linewidth=2)
plt.xlabel('Data Size (rows)', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Accuracy Comparison Across Data Sizes', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(results_df['data_size'])

plt.subplot(1, 2, 2)
plt.plot(results_df['data_size'], results_df['classical_svc_time'], 
         marker='o', label='Classical SVC', linewidth=2)
plt.plot(results_df['data_size'], results_df['qsvc_zz_time'], 
         marker='s', label='QSVC (ZZ Feature Map)', linewidth=2)
plt.plot(results_df['data_size'], results_df['qsvc_custom_time'], 
         marker='^', label='QSVC (Custom Feature Map)', linewidth=2)
plt.xlabel('Data Size (rows)', fontsize=12)
plt.ylabel('Training Time (seconds)', fontsize=12)
plt.title('Training Time Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(results_df['data_size'])
plt.yscale('log')  # Log scale for better visualization

plt.tight_layout()
plt.show()


## 9. Detailed Comparison Table


In [ ]:
# Detailed comparison table
comparison_df = pd.DataFrame({
    'Data Size': results_df['data_size'],
    'Classical SVC Acc': results_df['classical_svc_accuracy'].round(4),
    'QSVC ZZ Acc': results_df['qsvc_zz_accuracy'].round(4),
    'QSVC Custom Acc': results_df['qsvc_custom_accuracy'].round(4),
    'Classical Time (s)': results_df['classical_svc_time'].round(2),
    'QSVC ZZ Time (s)': results_df['qsvc_zz_time'].round(2),
    'QSVC Custom Time (s)': results_df['qsvc_custom_time'].round(2)
})

print("\nDetailed Comparison Table:")
print("="*100)
print(comparison_df.to_string(index=False))
print("="*100)


## 10. Performance Analysis


In [ ]:
# Calculate average improvements
avg_classical_acc = results_df['classical_svc_accuracy'].mean()
avg_zz_acc = results_df['qsvc_zz_accuracy'].mean()
avg_custom_acc = results_df['qsvc_custom_accuracy'].mean()

print("\n" + "="*80)
print("AVERAGE PERFORMANCE ACROSS ALL DATA SIZES")
print("="*80)
print(f"Classical SVC Average Accuracy:     {avg_classical_acc:.4f}")
print(f"QSVC (ZZ) Average Accuracy:         {avg_zz_acc:.4f}")
print(f"QSVC (Custom) Average Accuracy:     {avg_custom_acc:.4f}")
print("\nImprovement over Classical SVC:")
print(f"  QSVC (ZZ):         {((avg_zz_acc - avg_classical_acc) / avg_classical_acc * 100):.2f}%")
print(f"  QSVC (Custom):     {((avg_custom_acc - avg_classical_acc) / avg_classical_acc * 100):.2f}%")
print("="*80)
